# Project Title: OTP Verification System - S8083 Prashantsagar Uppara

# Importing required libraries

In [ ]:
from random import randrange
import smtplib
from email.mime.text import MIMEText
import time
import re
import PyQt6
from PyQt6.QtWidgets import QApplication, QWidget, QLabel, QLineEdit, QPushButton, QVBoxLayout, QMessageBox
from PyQt6.QtCore import QTimer, QTime
import sys

import warnings
warnings.filterwarnings('ignore')

# SMTP Server & Email Handling

In [ ]:
def send_email(receiver_email,a):
 # creating SMTP session
 s = smtplib.SMTP('smtp.outlook.com', 587)

 # starting TLS for security
 s.starttls()

 # Authentication credentials
 smtp_username = '###############@#################.com'
 smtp_password = '######################'

 s.login(smtp_username,smtp_password)

 # Email body
 msg = MIMEText(f"Your OTP for verification is: {a}")
 msg['Subject'] = 'OTP Verification'
 msg['From'] = smtp_username
 msg['To'] = receiver_email

 # sending the mail
 s.sendmail(smtp_username, receiver_email, msg.as_string())
 

 # terminating the session after use
 s.quit()

# OTP Generator

In [ ]:
# OTP generation function
def gen_otp():
 otp = randrange(100000, 999999)
 return otp

# Verification System

In [ ]:
def verifier():
    while True:
        #To check the input format of the user       
        user_input = input("Enter an email address: ")
        if validate_email_format(user_input):
            # To check valid email format
            b = gen_otp()
            send_email(user_input,b)
            checker(b)
            break  # Exiting the loop when a valid email is entered            
        else:
            print("Invalid email format. Please enter a valid email address.")

In [ ]:
# refernce to match the input format with user input
def validate_email_format(email):
    pattern = r'^.+@.+\..+$'
    return re.match(pattern, email) is not None

In [ ]:
# Actual code working without GUI
def checker(z):
    max_attempts = 3
    wait_time = 30  # delay time in seconds
    failed_attempts = 0
    
    while failed_attempts < max_attempts:
        
        otp_input = int(input("Enter OTP sent to your entered email address: "))                
        if otp_input == z: 
            
            print("Login Successful")
            break  # Exit the loop when a correct OTP is entered
        
        elif failed_attempts == 2:
         print(f"Maximum attempts reached. Please try again after {wait_time / 3} minutes.")
         break
            
        else:
            
            print(f"Invalid OTP. {max_attempts - failed_attempts - 1} attempt remaining. Wait 30 secs to retry ")
            time.sleep(wait_time)  # Wait for 30 seconds before the next attempt
        failed_attempts += 1 

In [ ]:
# Running the OTP generation, verification system & checking it in jupyter notebook
#verifier()

# PyQt6 GUI

In [ ]:
class LogInApp(QWidget):
    def __init__(self):
        super().__init__()
        self.initUI()
        self.otp = None
        self.setWindowTitle("💻 Log In")
        self.setGeometry(100, 100, 300, 250)
        self.failed_attempts = 0
            
        
    def initUI(self):
        # Making the layout
        layout = QVBoxLayout()

        # Adding email Label and textbox (LineEdit) for entering valid email
        self.email_label = QLabel('📨 Email')
        self.email_input = QLineEdit()
        layout.addWidget(self.email_label)
        layout.addWidget(self.email_input)
        

        # Adding send otp push button (also verifies email for valid entry)
        self.verify_email_btn = QPushButton('📤 Send OTP')
        self.verify_email_btn.clicked.connect(self.send_otp)
        layout.addWidget(self.verify_email_btn)
        
        # Adding otp Label and textbox (LineEdit) for entering sent otp over entered email
        self.otp_label = QLabel('🔢 OTP')
        self.otp_input = QLineEdit()
        layout.addWidget(self.otp_label)
        layout.addWidget(self.otp_input)

        
        # Adding verify OTP Button
        self.verify_otp_btn = QPushButton('🚦Verify OTP')
        self.verify_otp_btn.clicked.connect(self.verify_otp)
        layout.addWidget(self.verify_otp_btn)

        
        # login session label with timer
        self.timer_label = QLabel("Session ends in: 05:00")
        self.remaining_time = 300    # 5 minutes in seconds, defined session timing
        layout.addWidget(self.timer_label)
        self.timer = QTimer(self)
        self.timer.timeout.connect(self.update_timer)
        self.timer.start(1000)     # Updating time every second
        
        
        # Setting the layout
        self.setLayout(layout)

    # verifying the entered email & sending otp to valid entered email address    
    def send_otp(self):
        email = self.email_input.text()
        if validate_email_format(email):
            self.otp = gen_otp()
            send_email(email, self.otp)
            self.email_input.setDisabled(True)
            self.verify_email_btn.setDisabled(True)
            QMessageBox.information(self, 'OTP Sent', 'OTP has been sent to the email address')
        else:
            QMessageBox.warning(self, 'Invalid Email', 'Invalid email!!! Please enter a valid email address')

     
    # verifying the entered otp received over email           
    def verify_otp(self):
        try:
            user_otp = int(self.otp_input.text())
            if user_otp == self.otp:
                QMessageBox.information(self, 'Login Successful', 'Login Successful')
                self.close()  # Closing the form after successful login
            elif self.failed_attempts == 2:
                QMessageBox.critical(self, 'Login Failed', 'Maximum attempts reached. Please try again after 10 minutes')
                self.close()  # Closing the form after acknowledging the failure message
            else:
                self.failed_attempts += 1
                self.otp_input.clear()  # Clearing the entered OTP
                self.otp_input.setDisabled(True)  # Disabling the OTP input for 30 seconds
                QTimer.singleShot(30000, self.enable_otp_input)  # Enabling the OTP textbox after 30 seconds
                QMessageBox.warning(self, 'Invalid OTP', f'Invalid OTP. {3 - self.failed_attempts} attempt(s) remaining. Wait 30 secs to retry')
        
        except ValueError:
            QMessageBox.warning(self, 'Invalid Input', 'Please enter only integers for the OTP.')
            self.otp_input.clear()  # Clearing the entered OTP
            
        
    def enable_otp_input(self):
        self.otp_input.setDisabled(False)  # Enabling the OTP textbox after 30 seconds

  
    # calculating & updating the session timer
    def update_timer(self):
        self.remaining_time -= 1
        minutes = self.remaining_time // 60
        seconds = self.remaining_time % 60
        self.timer_label.setText(f"Session ends in: {minutes:02}:{seconds:02}")

        if self.remaining_time == 0:
            self.timer.stop()
            QMessageBox.critical(self, "Time Exceeded", "Session time exceeded. Please try again after 10 minutes")
            self.close()
    
    
# Main function to run the application
def main():
    app = QApplication(sys.argv)   # creating instance of QApplication
    ex = LogInApp()
    ex.show()
    sys.exit(app.exec())           # creates actial instance waits for user inputs all loops to execute and closes after the event

if __name__ == '__main__':         # execute the following code only if this file is run as a script from the command line
    main()